In [50]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


In [51]:
# DB接続
engine = create_engine('sqlite:///test.db')

Base = declarative_base()


In [52]:
import pprint

pprint.pprint(Base.metadata)

MetaData()


In [53]:
Base.metadata.create_all(engine)


In [54]:
# セッションの生成
Session = sessionmaker(bind=engine)
session = Session()


In [55]:
from sqlalchemy import MetaData

In [56]:
meta = MetaData(engine)
meta.reflect()

In [57]:
pprint.pprint(vars(meta)) # これで中身を確かめて自力でclassを作る？


{'_bind': Engine(sqlite:///test.db),
 '_fk_memos': defaultdict(<class 'list'>,
                          {('users', 'id'): [ForeignKey('users.id')]}),
 '_schemas': set(),
 '_sequences': {},
 'dispatch': <sqlalchemy.event.base.DDLEventsDispatch object at 0x0000016DC55042C0>,
 'naming_convention': immutabledict({'ix': 'ix_%(column_0_label)s'}),
 'schema': None,
 'tables': FacadeDict({'posts': Table('posts', MetaData(bind=Engine(sqlite:///test.db)), Column('id', INTEGER(), table=<posts>, primary_key=True, nullable=False), Column('users_id', INTEGER(), ForeignKey('users.id'), table=<posts>), Column('title', VARCHAR(), table=<posts>), Column('body', INTEGER(), table=<posts>), schema=None), 'users': Table('users', MetaData(bind=Engine(sqlite:///test.db)), Column('id', INTEGER(), table=<users>, primary_key=True), Column('name', VARCHAR(), table=<users>), Column('age', INTEGER(), table=<users>), schema=None)})}


In [58]:
# User
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer)
    id.primary_key = True
    name = Column(String)
    age = Column(Integer)

    posts = relationship('Post', backref='users')

In [60]:
# まずはPostから
class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer)
    id.primary_key = True
    users_id = Column(Integer, ForeignKey('users.id'))
    title = Column(String)
    body = Column(Integer)

    user = relationship('User')

In [61]:
# セッションの生成
Session = sessionmaker(bind=engine)
session = Session()


In [62]:
session.commit()


In [63]:
# inner joinのサンプル
users = session.query(User).join(Post, User.id == Post.users_id).all()


C:\Users\naoki\AppData\Local\Temp\ipykernel_19872\2938513795.py:2: SAWarning: relationship 'Post.user' will copy column users.id to column posts.users_id, which conflicts with relationship(s): 'Post.users' (copies users.id to posts.users_id), 'User.posts' (copies users.id to posts.users_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="posts,users"' to the 'Post.user' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
  users = session.query(User).join(Post, User.id == Post.users_id).all()


In [64]:
pprint.pprint(users)

In [65]:
user = users[0]

pprint.pprint(vars(user))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x0000016DC6264BB0>,
 'age': 19,
 'id': 1,
 'name': 'Suzuki'}


In [66]:
for user in users:
    print(f'{user.name}さんの投稿')
    for post in user.posts:
        print(f"|- タイトル: {post.title}")
    print()


Suzukiさんの投稿
|- タイトル: 朝の体操
|- タイトル: 今日の夕食
|- タイトル: 朝の体操
|- タイトル: 今日の夕食

Tanakaさんの投稿
|- タイトル: 仕事
|- タイトル: Python楽しい
|- タイトル: 仕事
|- タイトル: Python楽しい



In [67]:
# left outer joinのサンプル
users_posts = session.query(User, Post).outerjoin(
    Post, User.id == Post.users_id).all()

for user_post in users_posts:
    if user_post.Post is not None:
        print(f'{user_post.User.name}さんの投稿 タイトル: {user_post.Post.title}')
    else:
        print(f'{user_post.User.name}さんの投稿 なし')


Suzukiさんの投稿 タイトル: 今日の夕食
Suzukiさんの投稿 タイトル: 今日の夕食
Suzukiさんの投稿 タイトル: 朝の体操
Suzukiさんの投稿 タイトル: 朝の体操
Tanakaさんの投稿 タイトル: Python楽しい
Tanakaさんの投稿 タイトル: Python楽しい
Tanakaさんの投稿 タイトル: 仕事
Tanakaさんの投稿 タイトル: 仕事
Satoさんの投稿 なし
